### I Don't Wanna pay fucking fees using bot !

In [2]:
import json
import math
import time
import pytz
import secrets
import requests
import credentials
from web3 import Web3
from datetime import datetime
from eth_account import Account

### Helper functions

In [3]:
def get_abi(contract_address):
    #Get Contract ABI with Etherscan API
    api_key = ETHERSCAN_KEY
    url = 'https://api.etherscan.io/api'
    params = {
        'module': 'contract',
        'action': 'getabi',
        'address': contract_address,
        'apikey': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] != '1':
        print("Error: ", data['message'])
        return None
    else:
        abi = data['result']
        return abi

### Connect to Ethereum

In [8]:
INFURA_KEY = credentials.infura_key
ETHERSCAN_KEY = credentials.etherscan_key

RPC = {
    'mainnet': f'https://mainnet.infura.io/v3/{INFURA_KEY}',
    'goreli': f'https://goerli.infura.io/v3/{INFURA_KEY}',
    'mainnet_ws': f'wss://mainnet.infura.io/ws/v3/{INFURA_KEY}',
    'goreli_ws': f'wss://goerli.infura.io/ws/v3/{INFURA_KEY}',
}

provider = RPC['goreli_ws']
w3 = Web3(Web3.WebsocketProvider(provider))
w3.is_connected()

True

### Create account for signing transactions

In [5]:
account = Account.from_key(credentials.private_key)
account.address

'0x6B12B690Fc279958715C29AD65CCCB33900f0F0B'

### Simple Test

In [14]:
w3.eth.get_transaction_count(account.address)
transaction = {
    'from': account.address,
    'to': '',
    'value': 500000000000000000,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 21000,
    'maxFeePerGas': 2000000000,
    'maxPriorityFeePerGas': 1500000000,
    'chainId': 5,
}
signed = w3.eth.account.sign_transaction(transaction, credentials.private_key)
tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)
tx = w3.eth.get_transaction(tx_hash)

In [24]:
tx

AttributeDict({'accessList': [],
 'blockHash': None,
 'blockNumber': None,
 'chainId': 5,
 'from': '0x6B12B690Fc279958715C29AD65CCCB33900f0F0B',
 'gas': 200000,
 'gasPrice': 2000000000,
 'hash': HexBytes('0x38ac73a00429517d1b5eee9015737ac4e6f14f7d5d47f0f46c6fcbdae1decef8'),
 'input': '0x',
 'maxFeePerGas': 2000000000,
 'maxPriorityFeePerGas': 1000000000,
 'nonce': 429,
 'r': HexBytes('0x8a85ca5e60c434a26b9869e770cbeaca7ba7bd7fe57ab074dbd7a807b73041fe'),
 's': HexBytes('0x26ca04617e7c6d9555b02f66193ac17bda3449a6dbb18b26b1125d254bc8ea81'),
 'to': '0xD2C6259b5cAf917F510FccB5Bd83c9E78F423098',
 'transactionIndex': None,
 'type': 2,
 'v': 0,
 'value': 500000000000000000,
 'yParity': '0x0'})

### Interact with Uniswap V2 Router

In [5]:
UNISWAP_V2_FACTORY = Web3.to_checksum_address("0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f")
UNISWAP_V2_ROUTER = Web3.to_checksum_address("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D")

UNISWAP_V2_FACTORY_ABI = get_abi(UNISWAP_V2_FACTORY)
UNISWAP_V2_ROUTER_ABI = get_abi(UNISWAP_V2_ROUTER)

In [18]:
UNISWAP_V2_FACTORY_CONTRACT = w3.eth.contract(address=UNISWAP_V2_FACTORY, abi=UNISWAP_V2_FACTORY_ABI)
UNISWAP_V2_ROUTER_CONTRACT = w3.eth.contract(address=UNISWAP_V2_ROUTER, abi=UNISWAP_V2_ROUTER_ABI)
WETH = Web3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")
PEPE = Web3.to_checksum_address("0x6982508145454Ce325dDbE47a25d4ec3d2311933")

UNISWAP_V2_ROUTER_CONTRACT.all_functions()

'''
Buy Token:
swapExactETHForTokensSupportingFeeOnTransferTokens(uint256,address[],address,uint256)

Sell Token:
swapExactTokensForETHSupportingFeeOnTransferTokens(uint256,uint256,address[],address,uint256)

Rotate Token:
swapExactTokensForTokensSupportingFeeOnTransferTokens(uint256,uint256,address[],address,uint256)
'''


[<Function WETH()>,
 <Function addLiquidity(address,address,uint256,uint256,uint256,uint256,address,uint256)>,
 <Function addLiquidityETH(address,uint256,uint256,uint256,address,uint256)>,
 <Function factory()>,
 <Function getAmountIn(uint256,uint256,uint256)>,
 <Function getAmountOut(uint256,uint256,uint256)>,
 <Function getAmountsIn(uint256,address[])>,
 <Function getAmountsOut(uint256,address[])>,
 <Function quote(uint256,uint256,uint256)>,
 <Function removeLiquidity(address,address,uint256,uint256,uint256,address,uint256)>,
 <Function removeLiquidityETH(address,uint256,uint256,uint256,address,uint256)>,
 <Function removeLiquidityETHSupportingFeeOnTransferTokens(address,uint256,uint256,uint256,address,uint256)>,
 <Function removeLiquidityETHWithPermit(address,uint256,uint256,uint256,address,uint256,bool,uint8,bytes32,bytes32)>,
 <Function removeLiquidityETHWithPermitSupportingFeeOnTransferTokens(address,uint256,uint256,uint256,address,uint256,bool,uint8,bytes32,bytes32)>,
 <Function